In [ ]:
# Google Drive와 마운트
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Project Folder 연결
from os.path import join  

MY_GOOGLE_DRIVE_PATH = 'My Drive/je'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print(PROJECT_PATH)

/content/drive/My Drive/je


In [ ]:
%cd "{PROJECT_PATH}"
!ls

/content/drive/My Drive/je
data


In [ ]:
import pandas as pd
from pandas.io import gbq

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("ok")

ok


In [ ]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'amiable-nirvana-236805'
client = bigquery.Client(project=project_id)

# train = client.query('''
#   SELECT 
#     * 
#   FROM `amiable-nirvana-236805.jeju_data_ver1.201901_202003_train` 
#   LIMIT 1000
#   ''').to_dataframe()

train = client.query('''
SELECT * FROM `amiable-nirvana-236805.jeju_data_ver1.201901_202003_train` LIMIT 1000 ''').to_dataframe()

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Created wheel for pandas-profiling: filename=pandas_profiling-2.8.0-py2.py3-none-any.whl size=259932 sha256=1e35282b8958ff4e6bb868df4b738dcb8bbe1f793c91c2c9022923f59cbf19c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-v5ydsgoz/wheels/56/c2/dd/8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
Successfully built pandas-profiling


In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train, title='Pandas Profiling Report', explorative=True)
profile.to_notebook_iframe()

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.0.5
Numpy : 1.18.5
Scikit-Learn : 0.22.2.post1
Python 3.6.9


In [ ]:
def grap_year(data):
  data = str(data)
  return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
data = train.copy()
data = data.fillna('')
print(data['REG_YYMM'].head())

0    201901
1    201902
2    201904
3    201905
4    201906
Name: REG_YYMM, dtype: int64


In [ ]:
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
print(data[['REG_YYMM','year']].head())

   REG_YYMM  year
0    201901  2019
1    201902  2019
2    201904  2019
3    201905  2019
4    201906  2019


In [ ]:
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
print(data[['REG_YYMM','year','month']].head())

   REG_YYMM  year  month
0    201901  2019      1
1    201902  2019      2
2    201904  2019      4
3    201905  2019      5
4    201906  2019      6


In [ ]:
data = data.drop(['REG_YYMM'], axis=1)
data.head()

CARD_SIDO_NM CARD_CCG_NM STD_CLSS_NM HOM_SIDO_NM  ...     AMT  CNT  year  month
0           강원         양구군     체인화 편의점          강원  ...   76760   12  2019      1
1           서울         동작구      택시 운송업          강원  ...  323180  106  2019      2
2           강원         삼척시    서양식 음식점업          강원  ...  192900   22  2019      4
3           강원         강릉시    서양식 음식점업          강원  ...  501400   58  2019      5
4           서울         동작구      택시 운송업          강원  ...   19100    3  2019      6

[5 rows x 13 columns]

In [ ]:
# 데이터 정제
df = data.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
  if str(dtypes[column]) == 'object':
    encoder = LabelEncoder()
    encoder.fit(df[column])
    encoders[column] = encoder

df_num = df.copy()
for column in encoders.keys():
  encoder = encoders[column]
  df_num[column] = encoder.transform(df[column])

In [ ]:
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT','AMT','CNT'],axis=1)
train_target = np.log1p(train_num['AMT'])

In [ ]:
gbm = GradientBoostingRegressor(n_estimators=4000);
gbm.fit(train_features, train_target)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=4000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
from itertools import product

CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs, HOM_SIDO_NMs,AGEs, SEX_CTGO_CDs, FLCs,years,months]
temp = np.array(list(product(*comb_list)))
temp = pd.DataFrame(data=temp , columns=train_features.columns)

In [ ]:
temp.head(10)

CARD_SIDO_NM  STD_CLSS_NM  HOM_SIDO_NM  AGE  SEX_CTGO_CD  FLC  year  month
0             0            0            0    0            1    1  2020      4
1             0            0            0    0            1    1  2020      7
2             0            0            0    0            2    1  2020      4
3             0            0            0    0            2    1  2020      7
4             0            1            0    0            1    1  2020      4
5             0            1            0    0            1    1  2020      7
6             0            1            0    0            2    1  2020      4
7             0            1            0    0            2    1  2020      7
8             0            2            0    0            1    1  2020      4
9             0            2            0    0            1    1  2020      7

In [ ]:
pred = gbm.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred,0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]
temp = temp.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
temp.head(10)

REG_YYMM  CARD_SIDO_NM  STD_CLSS_NM       AMT
0    202004             0            0   75320.0
1    202004             0            1   38095.0
2    202004             0            2   24543.0
3    202004             0            3   91296.0
4    202004             0            4  157709.0
5    202004             0            5  436274.0
6    202004             0            6  527426.0
7    202004             0            7  777556.0
8    202004             0            8  243420.0
9    202004             0            9  576839.0

In [ ]:
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
print(temp.head())

   REG_YYMM CARD_SIDO_NM   STD_CLSS_NM       AMT
0    202004           강원       골프장 운영업   75320.0
1    202004           강원   과실 및 채소 소매업   38095.0
2    202004           강원  그외 기타 종합 소매업   24543.0
3    202004           강원  기타 대형 종합 소매업   91296.0
4    202004           강원   기타 외국식 음식점업  157709.0


In [ ]:
submission = client.query('''
SELECT * FROM `amiable-nirvana-236805.jeju_data_ver1.submission` ''').to_dataframe()

In [ ]:
submission = submission.drop(['AMT'],axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission['AMT'] = submission['AMT'].fillna(0)
print(submission.head())

   id  REG_YYMM CARD_SIDO_NM           STD_CLSS_NM      AMT
0   0    202004           강원            건강보조식품 소매업      0.0
1   1    202004           강원               골프장 운영업  75320.0
2   2    202004           강원           과실 및 채소 소매업  38095.0
3   3    202004           강원     관광 민예품 및 선물용품 소매업      0.0
4   4    202004           강원  그외 기타 분류안된 오락관련 서비스업      0.0


In [ ]:
submission.to_csv('submission.csv',encoding='utf-8-sig',index=False)